In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

df = pd.read_csv('/project_data/data_asset/Final_AR Data_shaped_058ec0b9.csv')

In [2]:
df = df[['CUSTOMER', 'NETDUE_DATE', 'CLEAR_DATE', 'CUST_CRE_REP_GRP', 'AMOUNT', 'ITEM_STATUS',
         'PAYMENT_TERMS', 'county', 'confirmed_cases', 'deaths']]

print(df)

        CUSTOMER NETDUE_DATE  CLEAR_DATE CUST_CRE_REP_GRP   AMOUNT  \
0         101374  2018-04-30  2018-02-07               AA   324.85   
1         101374  2018-04-30  2018-02-07               AA   324.85   
2         101374  2018-04-30  2018-02-07               AA   324.85   
3         101374  2018-04-30  2018-02-07               AA   481.49   
4         101374  2018-04-30  2018-02-07               AA   324.85   
5         101374  2018-04-30  2018-02-07               AA   324.85   
6         101374  2018-04-30  2018-02-07               AA   324.85   
7         101374  2018-04-30  2018-02-07               AA   881.10   
8         101374  2018-04-30  2018-02-07               AA   324.85   
9         101374  2018-04-30  2018-02-07               AA   643.47   
10        101374  2018-04-30  2018-02-07               AA   617.66   
11        101374  2018-04-30  2018-02-07               AA   324.85   
12        101374  2018-04-30  2018-02-07               AA   324.85   
13        101374  20

In [3]:
# Prelim pre-processing for target
df['NETDUE_DATE'] = df['NETDUE_DATE'].astype('datetime64[ns]')
df['CLEAR_DATE'] = df['CLEAR_DATE'].astype('datetime64[ns]')
df['DAY_PAID'] = df['NETDUE_DATE'] - df['CLEAR_DATE']
df['DAY_PAID'] = df['DAY_PAID'].astype('timedelta64[D]')
print(df)
print(df.dtypes)

df = df.loc[df['ITEM_STATUS'] == 'Closed']
print(df.columns)

        CUSTOMER NETDUE_DATE CLEAR_DATE CUST_CRE_REP_GRP  AMOUNT ITEM_STATUS  \
0         232752  2019-01-18 2019-01-18               BB   23.14      Closed   
1         232752  2019-03-01 2019-03-01               BB   23.14      Closed   
2         232752  2019-02-28 2019-02-28               BB   23.14      Closed   
3         232752  2019-02-22 2019-02-22               BB   23.14      Closed   
4         232752  2019-02-14 2019-02-14               BB   23.14      Closed   
5         232752  2019-03-14 2019-03-14               BB   23.14      Closed   
6         232752  2019-04-03 2019-04-03               BB   23.14      Closed   
7         232752  2019-04-04 2019-04-04               BB   23.14      Closed   
8         232752  2019-04-11 2019-04-11               BB   23.14      Closed   
9         232752  2019-04-15 2019-04-15               BB   23.14      Closed   
10        232752  2019-04-17 2019-04-17               BB   23.14      Closed   
11        232752  2019-04-17 2019-04-17 

Index(['CUSTOMER', 'NETDUE_DATE', 'CLEAR_DATE', 'CUST_CRE_REP_GRP', 'AMOUNT',
       'ITEM_STATUS', 'PAYMENT_TERMS', 'county', 'confirmed_cases', 'deaths',
       'DAY_PAID'],
      dtype='object')


In [4]:
# Label encoding for categorical variables
encode_cust = LabelEncoder()
encode_cust_rep = LabelEncoder()
encode_pay = LabelEncoder()

df['CUSTOMER'] = encode_cust.fit_transform(df['CUSTOMER'])
df['PAYMENT_TERMS'] = encode_pay.fit_transform(df['PAYMENT_TERMS'].astype(str))
df['CUST_CRE_REP_GRP'] = encode_cust_rep.fit_transform(df['CUST_CRE_REP_GRP'])

In [5]:
# Random Forest Regressor
df_imp = df[['CUSTOMER', 'CUST_CRE_REP_GRP', 'AMOUNT', 'PAYMENT_TERMS', 'confirmed_cases', 'DAY_PAID']]
X = df_imp[['CUSTOMER', 'CUST_CRE_REP_GRP', 'AMOUNT', 'PAYMENT_TERMS', 'confirmed_cases']]
y = df_imp['DAY_PAID']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# rf_clf = RandomForestRegressor(n_estimators=3, criterion="mae")
# rf_clf.fit(X_train, y_train)
# y_test_predicted = rf_clf.predict(X_test)
# print('Feature importances =', rf_clf.feature_importances_)
# print("R2 Score =", r2_score(y_test, y_test_predicted))
# print("Mean Absolute Error =", mean_absolute_error(y_test, y_test_predicted))

In [6]:
trainx = pd.DataFrame(X_train).reset_index(drop=True)
trainy = pd.DataFrame(y_train).reset_index(drop=True)
train_set = pd.concat([trainx, trainy], axis=1)

from project_lib import Project
project = Project.access()
project.save_data("AR_REG_TRAIN.csv", train_set.to_csv(index=False))

{'file_name': 'AR_REG_TRAIN.csv',
 'message': 'File saved to project storage.',
 'asset_id': '5165da46-b0ca-4d81-b1ea-264ff193afda'}

In [11]:
# Predictions on Open
pred_df = pd.read_csv('/project_data/data_asset/bq-results-20200505-181914-yglz5y8jrefn.csv')
pred_df2 = pd.read_csv('/project_data/data_asset/MI_County_Covid193.csv_shaped_21d54855.csv')
pred_df = pred_df.rename(columns={"custcounty": "county"})
pred_df = pred_df.merge(pred_df2, how='left', on='county')

pred_df = pred_df[['CUSTOMER', 'CustomerNm','FISCPER', 'NETDUE_DATE', 'CLEAR_DATE', 'CUST_CRE_REP_GRP', 'AMOUNT', 'ITEM_STATUS',
         'PAYMENT_TERMS', 'county', 'confirmed_cases', 'deaths']]
pred_df = pred_df.loc[pred_df['ITEM_STATUS'] == 'Open']
pred_df_2 = pred_df[['CUSTOMER', 'CUST_CRE_REP_GRP', 'AMOUNT', 'PAYMENT_TERMS', 'confirmed_cases']]

pred_df_2['CUSTOMER'] = encode_cust.transform(pred_df_2['CUSTOMER'])
pred_df_2['PAYMENT_TERMS'] = encode_pay.transform(pred_df_2['PAYMENT_TERMS'].astype(str))
pred_df_2['CUST_CRE_REP_GRP'] = encode_cust_rep.transform(pred_df_2['CUST_CRE_REP_GRP'])

pred_X = pred_df_2[['CUSTOMER', 'CUST_CRE_REP_GRP', 'AMOUNT', 'PAYMENT_TERMS', 'confirmed_cases']]
pred_y = rf_clf.predict(pred_X)
pred_df['Predictions'] = pred_y

# Post Open Transaction Predictions as csv
from project_lib import Project
project = Project.access()
project.save_data("AR_REGRESSION_PREDICTIONS.csv", pred_df.to_csv(index=False))

   COMP_CODE  CUSTOMER  FISCPER  DOC_NUMBER  LINE_ITEM  SUB_ITEM DOC_TYPE  \
0       1000    232752  2019001  9028989092          1         1       ZA   
1       1000    232752  2019002  9030204614          1         1       ZA   
2       1000    232752  2019002  9030165168          1         1       ZA   
3       1000    232752  2019002  9029983406          1         1       ZA   
4       1000    232752  2019002  9029722367          1         1       ZA   

  POSTING_DATE    DOC_DATE NETDUE_DATE  ... AMOUNT ITEM_STATUS PAYMENT_TERMS  \
0   2019-01-18  2019-01-18  2019-01-18  ...  23.14      Closed        NET000   
1   2019-03-01  2019-03-01  2019-03-01  ...  23.14      Closed        NET000   
2   2019-02-28  2019-02-28  2019-02-28  ...  23.14      Closed        NET000   
3   2019-02-22  2019-02-22  2019-02-22  ...  23.14      Closed        NET000   
4   2019-02-14  2019-02-14  2019-02-14  ...  23.14      Closed        NET000   

   GL_ACCOUNT __index_level_0__ CustomerId       Custome

In [ ]:
# # WML Authentication
# from watson_machine_learning_client import WatsonMachineLearningAPIClient
# wml_credentials = {
#                     "url": "https://default-cpd-default.miracle311-273183-3cfa7977c157a3e8d61067e13964bdf8-0001.us-south.containers.appdomain.cloud",
#                     "username": "miracle1",
#                     "password": "miracle1",
#                     "instance_id": "wml_local",
#                     "version" : "2.5.0"
#  }
# client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# # Set space
# client.set.default_space('e2c4bfa5-0762-4fe4-a942-edc59772f226')
# print(client.deployments.list())

In [ ]:
# meta_props={
#    client.repository.ModelMetaNames.NAME: "AR_COVID Closing Regression Prediction",
#    client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3.6",
#    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.20"     
#  }

In [ ]:
# model_artifact = client.repository.store_model(rf_clf,
#                                              meta_props=meta_props,
#                                              training_data=X_train,
#                                              training_target=y_train)
# model_uid = client.repository.get_model_uid(model_artifact)
# print("Model UID = " + model_uid)

In [ ]:
# model_details = client.repository.get_details(model_uid)

In [ ]:
# import json
# print(json.dumps(model_details, indent=2))

In [ ]:
# metaProps = {
# client.deployments.ConfigurationMetaNames.NAME: "AR_COVID_REG_DEPLOYMENT",
# client.deployments.ConfigurationMetaNames.ONLINE: {}
# }

In [ ]:
# created_deployment = client.deployments.create(model_uid, metaProps)

In [ ]:
# client.spaces.list()

In [1]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.supporting_classes import PayloadRecord

aios_credentials = {
                      "instance_guid": "00000000-0000-0000-0000-000000000000",
                      "username": "miracle1",
                      "password": "miracle1",
                      # address should be replaced with ip, port pair to be used in scripts outside ICP
                      "url": "https://default-cpd-default.miracle311-273183-3cfa7977c157a3e8d61067e13964bdf8-0001.us-south.containers.appdomain.cloud"
                   }

client = APIClient4ICP(aios_credentials)
subscription = client.data_mart.subscriptions.get(subscription_uid="cec75d63-76ae-4506-9735-abdc89e29d5b")


request_data = {
    "fields": ["CUSTOMER", "CUST_CRE_REP_GRP", "AMOUNT", "PAYMENT_TERMS", "confirmed_cases"],
    "values": [[2, 0, 229.62, 5, 311]]
  }
response_data = {
    "fields": ["CUSTOMER", "CUST_CRE_REP_GRP", "AMOUNT", "PAYMENT_TERMS", "confirmed_cases", "prediction"],
    "values": [[2, 0, 229.62, 5, 311, 64.0]]
  }

records = [PayloadRecord(request=request_data, response=response_data, response_time=18), 
                PayloadRecord(request=request_data, response=response_data, response_time=12)]

subscription.payload_logging.store(records=records)

/opt/conda/envs/Python-3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python-3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python-3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python-3.6/lib/python3.6/site-packages/urllib3/connectionpoo